# Testing everything

In [1]:
import pandas as pd
from datetime import datetime
from googlesearch import search
import time

In [2]:
fund_list_file = 'fundlist.txt'
df = pd.read_csv(fund_list_file, sep='\t',header=None,parse_dates=True,usecols=[1,3,4,8])
df = df.rename(columns={1:'name',3:'currency',4:'category',8:'risk'})

In [3]:
def mysearch(search_str:str):
    search_result = search(search_str, tld="com", num=20, stop=20, pause=5)
    return list(search_result)

dfi = pd.read_csv('fund_symbols.csv')
data = []
try:
    for i in range(391,400):
        x = None
        try:
            print("Searching for: ",df.loc[i]['name'])
            search_result = mysearch(df.loc[i]['name']+" site:morningstar.pt")
            print(search_result)
        except Exception as e:
            print("Search Failed")
            raise e
        for url in search_result:
            if url[:58] == "https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx":
                url = url[:72]
                x = pd.read_html(url)[3]
                break
        if x is None:
            search_result = mysearch(df.loc[i]['name']+" site:morningstar.es")
            print(search_result)
            for url in search_result:
                if url[:58] == "https://www.morningstar.es/es/funds/snapshot/snapshot.aspx":
                    url = url[:72]
                    x = pd.read_html(url)[3]
                    break
        if x is None:
            print("'{}' Not Found!".format(df.loc[i]['name']))
        else:
            symbol = x.loc[4][2]+":"+x.loc[1][2][:3]
            print("{} {} '{}' {}".format(i, symbol,df.loc[i]['name'],url))
            data.append((i, symbol,df.loc[i]['name'],df.loc[i]['currency'],df.loc[i]['risk'],df.loc[i]['category'],url))
except Exception as e:
    print(e)
    print('line {}'.format(i))
    
dfd = pd.DataFrame(data=data,columns=['ID','symbol','name','currency','risk','category','url'])
pd.concat([dfi,dfd]).to_csv('fund_symbols.csv',index=False)

Searching for:  Fidelity Emerging Mkts Foc A-Acc-USD
['https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx?id=F00000UFI4']
list index out of range
line 391
